In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [2]:
class DiffusionModel(nn.Module):
    def __init__(self):
        super(DiffusionModel, self).__init__()

        # 卷积层
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # 全连接层
        self.fc1 = nn.Sequential(
            nn.Linear(32 * 8 * 8, 128),
            nn.ReLU()
        )

        self.fc2 = nn.Linear(128, 3 * 32 * 32)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        x = x.view(-1, 3, 32, 32)
        return x




In [3]:
# 实例化网络
model = DiffusionModel()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [4]:
# 加载数据并划分为训练集、验证集和测试集
data = np.load("data4D.npy")
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# 将NumPy数组转换为张量
train_data_tensor = torch.from_numpy(train_data)
val_data_tensor = torch.from_numpy(val_data)
test_data_tensor = torch.from_numpy(test_data)

# 创建DataLoader
batch_size = 128
train_dataset = TensorDataset(train_data_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(val_data_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(test_data_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# 训练模型
num_epochs = 250
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DiffusionModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        inputs = data[0].to(device)
        targets = data[0].to(device) # 为了重构输入，使用输入数据作为目标数据

        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item()}')

    # 验证模型
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, data in enumerate(val_loader):
            inputs = data[0].to(device)
            targets = data[0].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss/len(val_loader)}')

Epoch [1/250], Step [10/25], Loss: 0.14247357845306396
Epoch [1/250], Step [20/25], Loss: 0.058277081698179245
Epoch [1/250], Validation Loss: 0.05218342372349331
Epoch [2/250], Step [10/25], Loss: 0.05281175300478935
Epoch [2/250], Step [20/25], Loss: 0.0505032017827034
Epoch [2/250], Validation Loss: 0.05128518545201847
Epoch [3/250], Step [10/25], Loss: 0.052752986550331116
Epoch [3/250], Step [20/25], Loss: 0.05458296090364456
Epoch [3/250], Validation Loss: 0.05084290515099253
Epoch [4/250], Step [10/25], Loss: 0.050522156059741974
Epoch [4/250], Step [20/25], Loss: 0.05187498778104782
Epoch [4/250], Validation Loss: 0.050300018063613346
Epoch [5/250], Step [10/25], Loss: 0.04999318718910217
Epoch [5/250], Step [20/25], Loss: 0.04962517321109772
Epoch [5/250], Validation Loss: 0.04993411526083946
Epoch [6/250], Step [10/25], Loss: 0.050143998116254807
Epoch [6/250], Step [20/25], Loss: 0.05115341767668724
Epoch [6/250], Validation Loss: 0.04971126413771084
Epoch [7/250], Step [10/

In [9]:
# 测试模型
model.eval()
test_loss = 0.0
with torch.no_grad():
    for batch_idx, data in enumerate(test_loader):
        inputs = data[0].to(device)
        targets = data[0].to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

print(f'Test Loss: {test_loss/len(test_loader)}')

Test Loss: 0.0356166479177773
